In [36]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
train = pd.read_csv('data/train.csv.zip', index_col='id')
train.head()

,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
id,,,,,,,,,,,,,,,,,,,,,
1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [34]:
test = pd.read_csv('data/test.csv.zip', index_col='id')
test.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
id,,,,,,,,,,,,,,,,,,,,,
4,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,0.023438,...,0.006836,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711
7,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,0.023438,...,0.000000,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922
9,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,0.005859,...,0.128910,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000
12,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,0.001953,...,0.012695,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789
13,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,0.000000,...,0.000000,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812


In [3]:
le = LabelEncoder().fit(train.species)
labels = le.transform(train.species)
le.classes_

array(['Acer_Capillipes', 'Acer_Circinatum', 'Acer_Mono', 'Acer_Opalus',
       'Acer_Palmatum', 'Acer_Pictum', 'Acer_Platanoids', 'Acer_Rubrum',
       'Acer_Rufinerve', 'Acer_Saccharinum', 'Alnus_Cordata',
       'Alnus_Maximowiczii', 'Alnus_Rubra', 'Alnus_Sieboldiana',
       'Alnus_Viridis', 'Arundinaria_Simonii', 'Betula_Austrosinensis',
       'Betula_Pendula', 'Callicarpa_Bodinieri', 'Castanea_Sativa',
       'Celtis_Koraiensis', 'Cercis_Siliquastrum', 'Cornus_Chinensis',
       'Cornus_Controversa', 'Cornus_Macrophylla', 'Cotinus_Coggygria',
       'Crataegus_Monogyna', 'Cytisus_Battandieri',
       'Eucalyptus_Glaucescens', 'Eucalyptus_Neglecta',
       'Eucalyptus_Urnigera', 'Fagus_Sylvatica', 'Ginkgo_Biloba',
       'Ilex_Aquifolium', 'Ilex_Cornuta', 'Liquidambar_Styraciflua',
       'Liriodendron_Tulipifera', 'Lithocarpus_Cleistocarpus',
       'Lithocarpus_Edulis', 'Magnolia_Heptapeta', 'Magnolia_Salicifolia',
       'Morus_Nigra', 'Olea_Europaea', 'Phildelphus', 'Populus_

In [4]:
train.describe()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
count,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,...,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000
mean,0.017412,0.028539,0.031988,0.023280,0.014264,0.038579,0.019202,0.001083,0.007167,0.018639,...,0.036501,0.005024,0.015944,0.011586,0.016108,0.014017,0.002688,0.020291,0.008989,0.019420
std,0.019739,0.038855,0.025847,0.028411,0.018390,0.052030,0.017511,0.002743,0.008933,0.016071,...,0.063403,0.019321,0.023214,0.025040,0.015335,0.060151,0.011415,0.039040,0.013791,0.022768
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.001953,0.001953,0.013672,0.005859,0.001953,0.000000,0.005859,0.000000,0.001953,0.005859,...,0.000000,0.000000,0.000977,0.000000,0.004883,0.000000,0.000000,0.000000,0.000000,0.000977
50%,0.009766,0.011719,0.025391,0.013672,0.007812,0.015625,0.015625,0.000000,0.005859,0.015625,...,0.004883,0.000000,0.005859,0.000977,0.012695,0.000000,0.000000,0.003906,0.002930,0.011719
75%,0.025391,0.041016,0.044922,0.029297,0.017578,0.056153,0.029297,0.000000,0.007812,0.027344,...,0.043701,0.000000,0.022217,0.009766,0.021484,0.000000,0.000000,0.023438,0.012695,0.029297
max,0.087891,0.205080,0.156250,0.169920,0.111330,0.310550,0.091797,0.031250,0.076172,0.097656,...,0.429690,0.202150,0.172850,0.200200,0.106450,0.578130,0.151370,0.375980,0.086914,0.141600


In [7]:
train.groupby('species').size()

species
Acer_Capillipes                 10
Acer_Circinatum                 10
Acer_Mono                       10
Acer_Opalus                     10
Acer_Palmatum                   10
Acer_Pictum                     10
Acer_Platanoids                 10
Acer_Rubrum                     10
Acer_Rufinerve                  10
Acer_Saccharinum                10
Alnus_Cordata                   10
Alnus_Maximowiczii              10
Alnus_Rubra                     10
Alnus_Sieboldiana               10
Alnus_Viridis                   10
Arundinaria_Simonii             10
Betula_Austrosinensis           10
Betula_Pendula                  10
Callicarpa_Bodinieri            10
Castanea_Sativa                 10
Celtis_Koraiensis               10
Cercis_Siliquastrum             10
Cornus_Chinensis                10
Cornus_Controversa              10
Cornus_Macrophylla              10
Cotinus_Coggygria               10
Crataegus_Monogyna              10
Cytisus_Battandieri             10
Eucalyptus_G

In [19]:
X_train = train.drop(['species'], axis=1)
y_train = labels

In [25]:
def knn(X, y):
    clf = KNeighborsClassifier()
    params = {
        'n_neighbors': [3, 5, 7, 10], 
        'weights': ['uniform', 'distance'], 
        'algorithm': ['ball_tree', 'kd_tree', 'brute']}
    grid_search = GridSearchCV(clf, param_grid=params, cv=StratifiedKFold(3, random_state=42), scoring='neg_log_loss')
    grid_search.fit(X, y)
    print('Best score: {}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    return grid_search.best_params_

knn_params = knn(X_train, y_train)

Best score: -1.2599850719081187
Best parameters: {'algorithm': 'ball_tree', 'weights': 'distance', 'n_neighbors': 7}


In [33]:
def rfc(X, y):
    clf = RandomForestClassifier()
    params = {'n_estimators' : [50, 100, 200]}
    grid_search = GridSearchCV(clf, param_grid=params, cv=StratifiedKFold(3, random_state=42), scoring='neg_log_loss')
    grid_search.fit(X, y)
    print('Best score: {}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    return grid_search.best_params_

rfc_params = rfc(X_train, y_train)

Best score: -0.8498889357970243
Best parameters: {'n_estimators': 200}


In [38]:
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)
pred = clf.predict_proba(test)

In [47]:
submission = pd.DataFrame(pred, columns=le.classes_, index=test.index)
submission.head()

,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,Acer_Saccharinum,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
id,,,,,,,,,,,,,,,,,,,,,
4,0.000,0.005,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,...,0.0,0.00,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.000
7,0.000,0.000,0.030,0.030,0.000,0.005,0.075,0.000,0.000,0.005,...,0.0,0.02,0.000,0.0,0.0,0.005,0.000,0.025,0.0,0.000
9,0.000,0.685,0.005,0.000,0.035,0.005,0.000,0.005,0.010,0.015,...,0.0,0.00,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.010
12,0.000,0.030,0.000,0.005,0.000,0.000,0.005,0.005,0.020,0.005,...,0.0,0.00,0.005,0.0,0.0,0.050,0.065,0.000,0.0,0.005
13,0.015,0.020,0.000,0.000,0.000,0.000,0.000,0.005,0.075,0.020,...,0.0,0.00,0.000,0.0,0.0,0.000,0.020,0.000,0.0,0.005


In [48]:
submission.to_csv('submission/rfc.csv', index_label='id')

In [50]:
def describeImportance(clf, X):
    indices = np.argsort(clf.feature_importances_)[::-1]
    for f in range(X.shape[1]):
        print('%d. feature %d %s (%f)' % (f + 1, indices[f], X.columns[indices[f]],
                                          clf.feature_importances_[indices[f]]))

In [52]:
describeImportance(clf, X_train)

1. feature 47 margin48 (0.010581)
2. feature 12 margin13 (0.008921)
3. feature 112 shape49 (0.008702)
4. feature 58 margin59 (0.008389)
5. feature 5 margin6 (0.008013)
6. feature 81 shape18 (0.007940)
7. feature 42 margin43 (0.007934)
8. feature 49 margin50 (0.007869)
9. feature 133 texture6 (0.007860)
10. feature 2 margin3 (0.007764)
11. feature 140 texture13 (0.007729)
12. feature 79 shape16 (0.007728)
13. feature 136 texture9 (0.007682)
14. feature 157 texture30 (0.007633)
15. feature 54 margin55 (0.007617)
16. feature 1 margin2 (0.007475)
17. feature 135 texture8 (0.007427)
18. feature 20 margin21 (0.007418)
19. feature 186 texture59 (0.007388)
20. feature 17 margin18 (0.007280)
21. feature 80 shape17 (0.007227)
22. feature 165 texture38 (0.007212)
23. feature 147 texture20 (0.007210)
24. feature 44 margin45 (0.007143)
25. feature 176 texture49 (0.006967)
26. feature 113 shape50 (0.006960)
27. feature 40 margin41 (0.006859)
28. feature 111 shape48 (0.006822)
29. feature 141 texture